In [37]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import math
import os

def load_data():
    """Load city coordinates and area assignments from CSV files"""
    
    # Load city coordinates
    cities_df = pd.read_csv('cities_data.csv')
    coords = {}
    for _, row in cities_df.iterrows():
        coords[row['city']] = (row['x'], row['y'])
    
    # Load area assignments
    areas_df = pd.read_csv('areas_data.csv')
    C1 = set(areas_df[areas_df['area'] == 'N1']['city'].tolist())
    C2 = set(areas_df[areas_df['area'] == 'N2']['city'].tolist())
    C3 = set(areas_df[areas_df['area'] == 'N3']['city'].tolist())
    
    return coords, C1, C2, C3

def euclid(a, b, coords):
    """Calculate Euclidean distance between two cities"""
    ax, ay = coords[a]
    bx, by = coords[b]
    return math.hypot(ax - bx, ay - by)

def solve_telecom_path():
    """Main optimization function following the exact formulation"""
    
    # Load data
    coords, C1, C2, C3 = load_data()
    
    # Define start and end cities for each area (C_start,k and C_slutt,k)
    C_start = {
        1: "Warsaw",
        2: "Kielce", 
        3: "Opole"
    }
    
    C_slutt = {
        1: "Radom",
        2: "Gliwice",
        3: "Jelenia Góra"
    }
    
    # NCITY_k parameters (number of arcs needed)
    NCITY = {
        1: 3,  # Area 1: 4 cities = 3 arcs
        2: 4,  # Area 2: 5 cities = 4 arcs
        3: 4   # Area 3: 5 cities = 4 arcs
    }
    
    # Create model
    m = gp.Model("Telecom_Path_SW_Poland")
    m.setParam('OutputFlag', 1)
    
    # Decision variables x_ij - only for connections within each area
    x = {}
    
    # Create variables for each area (no self-loops)
    for i in C1:
        for j in C1:
            if i != j:  # x_ii = 0 implemented here
                x[i, j] = m.addVar(vtype=GRB.BINARY, name=f"x_{i}_{j}")
    
    for i in C2:
        for j in C2:
            if i != j:
                x[i, j] = m.addVar(vtype=GRB.BINARY, name=f"x_{i}_{j}")
    
    for i in C3:
        for j in C3:
            if i != j:
                x[i, j] = m.addVar(vtype=GRB.BINARY, name=f"x_{i}_{j}")
    
    # Objective: minimize total distance
    obj = gp.LinExpr()
    for (i, j) in x:
        obj += euclid(i, j, coords) * x[i, j]
    m.setObjective(obj, GRB.MINIMIZE)
    
    # CONSTRAINTS
    
    # 1) Start city constraints for each area
    # Network starts at the start city (one outgoing arc)
    m.addConstr(gp.quicksum(x["Warsaw", j] for j in C1 if j != "Warsaw") == 1, "start_out_1")
    m.addConstr(gp.quicksum(x["Kielce", j] for j in C2 if j != "Kielce") == 1, "start_out_2")
    m.addConstr(gp.quicksum(x["Opole", j] for j in C3 if j != "Opole") == 1, "start_out_3")
    
    # Network cannot enter the start city
    m.addConstr(gp.quicksum(x[i, "Warsaw"] for i in C1 if i != "Warsaw") == 0, "start_in_1")
    m.addConstr(gp.quicksum(x[i, "Kielce"] for i in C2 if i != "Kielce") == 0, "start_in_2")
    m.addConstr(gp.quicksum(x[i, "Opole"] for i in C3 if i != "Opole") == 0, "start_in_3")
    
    # 2) End city constraints for each area
    # Network ends at the end city (one incoming arc)
    m.addConstr(gp.quicksum(x[i, "Radom"] for i in C1 if i != "Radom") == 1, "end_in_1")
    m.addConstr(gp.quicksum(x[i, "Gliwice"] for i in C2 if i != "Gliwice") == 1, "end_in_2")
    m.addConstr(gp.quicksum(x[i, "Jelenia Góra"] for i in C3 if i != "Jelenia Góra") == 1, "end_in_3")
    
    # Network cannot exit the end city
    m.addConstr(gp.quicksum(x["Radom", j] for j in C1 if j != "Radom") == 0, "end_out_1")
    m.addConstr(gp.quicksum(x["Gliwice", j] for j in C2 if j != "Gliwice") == 0, "end_out_2")
    m.addConstr(gp.quicksum(x["Jelenia Góra", j] for j in C3 if j != "Jelenia Góra") == 0, "end_out_3")
    
    # 3) Network cannot split (at most one in and one out for each city)
    for city in C1:
        m.addConstr(gp.quicksum(x[city, j] for j in C1 if j != city and (city, j) in x) <= 1, f"out_max_{city}")
        m.addConstr(gp.quicksum(x[i, city] for i in C1 if i != city and (i, city) in x) <= 1, f"in_max_{city}")
    
    for city in C2:
        m.addConstr(gp.quicksum(x[city, j] for j in C2 if j != city and (city, j) in x) <= 1, f"out_max_{city}")
        m.addConstr(gp.quicksum(x[i, city] for i in C2 if i != city and (i, city) in x) <= 1, f"in_max_{city}")
    
    for city in C3:
        m.addConstr(gp.quicksum(x[city, j] for j in C3 if j != city and (city, j) in x) <= 1, f"out_max_{city}")
        m.addConstr(gp.quicksum(x[i, city] for i in C3 if i != city and (i, city) in x) <= 1, f"in_max_{city}")
    
    # 4) Lower bound for number of arcs in each area
    m.addConstr(gp.quicksum(x[i, j] for i in C1 for j in C1 if (i, j) in x) >= NCITY[1], "ncity_1")
    m.addConstr(gp.quicksum(x[i, j] for i in C2 for j in C2 if (i, j) in x) >= NCITY[2], "ncity_2")
    m.addConstr(gp.quicksum(x[i, j] for i in C3 for j in C3 if (i, j) in x) >= NCITY[3], "ncity_3")
    
    # 5) No loops allowed (no bidirectional connections)
    for i in C1 | C2 | C3:
        for j in C1 | C2 | C3:
            if i < j and (i, j) in x and (j, i) in x:
                m.addConstr(x[i, j] + x[j, i] <= 1, f"no_loop_{i}_{j}")
    
    # 6) Node balance - flow conservation for intermediate cities
    for city in C1 - {"Warsaw", "Radom"}:
        m.addConstr(
            gp.quicksum(x[i, city] for i in C1 if i != city and (i, city) in x) -
            gp.quicksum(x[city, j] for j in C1 if j != city and (city, j) in x) == 0,
            f"balance_{city}"
        )
    
    for city in C2 - {"Kielce", "Gliwice"}:
        m.addConstr(
            gp.quicksum(x[i, city] for i in C2 if i != city and (i, city) in x) -
            gp.quicksum(x[city, j] for j in C2 if j != city and (city, j) in x) == 0,
            f"balance_{city}"
        )
    
    for city in C3 - {"Opole", "Jelenia Góra"}:
        m.addConstr(
            gp.quicksum(x[i, city] for i in C3 if i != city and (i, city) in x) -
            gp.quicksum(x[city, j] for j in C3 if j != city and (city, j) in x) == 0,
            f"balance_{city}"
        )
    
    # 7) POLICY CONSTRAINTS (following the exact formulation)
    
    # At least one of Konin (C1) or Kalisz (C3)
    konin_sum = gp.quicksum(x[i, "Konin"] for i in C1 if i != "Konin" and (i, "Konin") in x)
    kalisz_sum = gp.quicksum(x[i, "Kalisz"] for i in C3 if i != "Kalisz" and (i, "Kalisz") in x)
    m.addConstr(konin_sum + kalisz_sum >= 1, "konin_kalisz")
    
    # At least one of Poznań, Zielona Góra, Leszno (all in C3)
    poznan_sum = gp.quicksum(x[i, "Poznań"] for i in C3 if i != "Poznań" and (i, "Poznań") in x)
    zielona_sum = gp.quicksum(x[i, "Zielona Góra"] for i in C3 if i != "Zielona Góra" and (i, "Zielona Góra") in x)
    leszno_sum = gp.quicksum(x[i, "Leszno"] for i in C3 if i != "Leszno" and (i, "Leszno") in x)
    m.addConstr(poznan_sum + zielona_sum + leszno_sum >= 1, "german_connection")
    
    # At most one of Bytom, Sosnowiec, Katowice (all in C2)
    bytom_sum = gp.quicksum(x[i, "Bytom"] for i in C2 if i != "Bytom" and (i, "Bytom") in x)
    sosnowiec_sum = gp.quicksum(x[i, "Sosnowiec"] for i in C2 if i != "Sosnowiec" and (i, "Sosnowiec") in x)
    katowice_sum = gp.quicksum(x[i, "Katowice"] for i in C2 if i != "Katowice" and (i, "Katowice") in x)
    m.addConstr(bytom_sum + sosnowiec_sum + katowice_sum <= 1, "decentralization")
    
    # At most one of Wodzisław Śląski and Jastrzębie-Zdrój (both in C2)
    slaski_sum = gp.quicksum(x[i, "Wodzisław Śląski"] for i in C2 if i != "Wodzisław Śląski" and (i, "Wodzisław Śląski") in x)
    jastrzebie_sum = gp.quicksum(x[i, "Jastrzębie-Zdrój"] for i in C2 if i != "Jastrzębie-Zdrój" and (i, "Jastrzębie-Zdrój") in x)
    m.addConstr(slaski_sum + jastrzebie_sum <= 1, "slaski_jastrzebie")
    
    # Solve
    print("Optimizing telecommunications network...")
    m.optimize()
    
    # Extract and display results
    if m.status == GRB.OPTIMAL:
        print("\n" + "="*60)
        print("OPTIMAL SOLUTION FOUND")
        print("="*60)
        
        total_length = m.objVal
        
        # Reconstruct paths for each area
        def reconstruct_path(area_cities, start, end):
            """Reconstruct path from start to end within an area"""
            path = [start]
            current = start
            visited = {start}
            
            while current != end:
                found_next = False
                for j in area_cities:
                    if j not in visited and (current, j) in x and x[current, j].X > 0.5:
                        path.append(j)
                        visited.add(j)
                        current = j
                        found_next = True
                        break
                
                if not found_next:
                    print(f"Warning: Path reconstruction failed at {current}")
                    break
            
            return path
        
        # Get paths for each area
        path1 = reconstruct_path(C1, "Warsaw", "Radom")
        path2 = reconstruct_path(C2, "Kielce", "Gliwice")
        path3 = reconstruct_path(C3, "Opole", "Jelenia Góra")
        
        print(f"\nTotal distance: {total_length:.3f}")
        
        # Print paths in the solution format
        print("\nMain line path:")
        
        # Area 1
        for i in range(len(path1) - 1):
            print(f"{path1[i]} - {path1[i+1]}")
        
        # Area 2
        for i in range(len(path2) - 1):
            print(f"{path2[i]} - {path2[i+1]}")
        
        # Area 3
        for i in range(len(path3) - 1):
            print(f"{path3[i]} - {path3[i+1]}")
        
        print("\n" + "="*60)
        print("NETWORK PATHS BY AREA")
        print("="*60)
        
        print(f"\nArea 1: {' – '.join(path1)}")
        print(f"Area 2: {' – '.join(path2)}")
        print(f"Area 3: {' – '.join(path3)}")
        
        # Verify constraints
        print("\n" + "="*60)
        print("CONSTRAINT VERIFICATION")
        print("="*60)
        
        print(f"\nArea coverage:")
        print(f"  Area 1: {len(path1)} cities (required: 4)")
        print(f"  Area 2: {len(path2)} cities (required: 5)")
        print(f"  Area 3: {len(path3)} cities (required: 5)")
        
        print(f"\nPolicy constraints:")
        if "Konin" in path1:
            print(f"  ✓ Konin included")
        elif "Kalisz" in path3:
            print(f"  ✓ Kalisz included")
        else:
            print(f"  ✗ Neither Konin nor Kalisz included")
        
        k2_cities = ["Bytom", "Sosnowiec", "Katowice"]
        included_k2 = [c for c in k2_cities if c in path2]
        if len(included_k2) <= 1:
            print(f"  ✓ Decentralization: {len(included_k2)} city from {k2_cities}")
        else:
            print(f"  ✗ Decentralization violated: {included_k2}")
        
        if "Wodzisław Śląski" in path2 and "Jastrzębie-Zdrój" in path2:
            print(f"  ✗ Both Wodzisław Śląski and Jastrzębie-Zdrój included")
        else:
            print(f"  ✓ Regional balance satisfied")
        
        k4_cities = ["Poznań", "Zielona Góra", "Leszno"]
        included_k4 = [c for c in k4_cities if c in path3]
        if included_k4:
            print(f"  ✓ German connection: {', '.join(included_k4)} included")
        else:
            print(f"  ✗ No German connection")
        
        return (path1, path2, path3), total_length
        
    else:
        print(f"\nOptimization failed. Status: {m.status}")
        if m.status == GRB.INFEASIBLE:
            print("The problem is infeasible - no solution satisfies all constraints.")
            m.computeIIS()
            m.write("model.ilp")
            print("Infeasible constraint set written to model.ilp")
        return None, None

if __name__ == "__main__":
    # Check if data files exist
    required_files = ['cities_data.csv', 'areas_data.csv']
    for file in required_files:
        if not os.path.exists(file):
            print(f"Error: Required data file '{file}' not found!")
            print("Please ensure all data files are in the same directory as this script.")
            exit(1)
    
    # Solve the optimization problem
    paths, length = solve_telecom_path()

Set parameter OutputFlag to value 1
Optimizing telecommunications network...
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 23.6.0 23H626)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 193 rows, 204 columns and 1307 nonzeros
Model fingerprint: 0x85ca3b23
Variable types: 0 continuous, 204 integer (204 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 267.4522983
Presolve removed 118 rows and 116 columns
Presolve time: 0.02s
Presolved: 75 rows, 88 columns, 493 nonzeros
Found heuristic solution: objective 243.8175348
Variable types: 0 continuous, 88 integer (88 binary)

Root relaxation: objective 1.241697e+02, 21 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |

In [41]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import math
import numpy as np
import sys
import os

# Set UTF-8 encoding for the environment
os.environ['PYTHONIOENCODING'] = 'utf-8'
if sys.platform == 'win32':
    import locale
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

def load_data(filename='cities_data.csv'):
    """Load city data from CSV file"""
    # Read the CSV file
    cities_df = pd.read_csv(filename)
    
    # Extract coordinates and capacities
    coords = {}
    capacities = {}
    
    for _, row in cities_df.iterrows():
        city = row['city']
        coords[city] = (row['x'], row['y'])
        capacities[city] = row['capacity_tb']
    
    return coords, capacities

def euclid(city_a, city_b, coords):
    """Calculate Euclidean distance between two cities"""
    x1, y1 = coords[city_a]
    x2, y2 = coords[city_b]
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

def clean_name_for_constraint(name):
    """Convert city name to ASCII for constraint names"""
    # Replace Polish characters with ASCII equivalents
    replacements = {
        'ą': 'a', 'ć': 'c', 'ę': 'e', 'ł': 'l', 'ń': 'n', 
        'ó': 'o', 'ś': 's', 'ź': 'z', 'ż': 'z',
        'Ą': 'A', 'Ć': 'C', 'Ę': 'E', 'Ł': 'L', 'Ń': 'N',
        'Ó': 'O', 'Ś': 'S', 'Ź': 'Z', 'Ż': 'Z',
        ' ': '_', '-': '_'
    }
    clean = name
    for polish, ascii_char in replacements.items():
        clean = clean.replace(polish, ascii_char)
    return clean

def solve_task2_switching_stations():
    """
    Solve Task 2: Connect remaining cities to switching stations
    Using the main line from Task 1 solution
    """
    
    # Load city data
    coords, capacities = load_data('cities_data.csv')
    
    # CORRECTED Main line cities from Task 1 solution
    # Based on the optimal solution with total distance 130.998
    main_line_cities = [
        # Area 1: Warsaw → Skierniewice → Piotrków → Radom
        "Warsaw", "Skierniewice", "Piotrków", "Radom",
        # Area 2: Kielce → Kraków → Bielsko → Jastrzębie-Zdrój → Gliwice
        "Kielce", "Kraków", "Bielsko", "Jastrzębie-Zdrój", "Gliwice",
        # Area 3: Opole → Kalisz → Leszno → Legnica → Jelenia Góra
        "Opole", "Kalisz", "Leszno", "Legnica", "Jelenia Góra"
    ]
    
    # Define sets
    N = set(coords.keys())  # All cities
    main_line_set = set(main_line_cities)
    R = N - main_line_set  # Remaining cities not on main line
    S = {"Warsaw", "Kielce", "Opole"}  # Switching stations
    
    print("="*60)
    print("TASK 2: CONNECTING REMAINING CITIES TO SWITCHING STATIONS")
    print("="*60)
    print(f"\nTotal cities: {len(N)}")
    print(f"Cities on main line: {len(main_line_set)}")
    print(f"Remaining cities to connect: {len(R)}")
    print(f"\nMain line cities: {sorted(main_line_set)}")
    print(f"\nRemaining cities: {sorted(R)}")
    print(f"Switching stations: {sorted(S)}")
    
    # Parameters
    alpha = 1000  # Cost per coordinate unit per TB/s (EUR)
    beta = 200    # Maximum capacity per cable (TB/s)
    
    # Switching station capacities (TB/s)
    C = {
        "Warsaw": 800,
        "Kielce": 1200,
        "Opole": 400
    }
    
    # City capacity requirements for remaining cities
    c = {city: capacities[city] for city in R}
    
    # Verify all remaining cities have capacity requirements
    print("\nCapacity requirements for remaining cities:")
    total_demand = 0
    for city in sorted(R):
        print(f"  {city}: {c[city]} TB/s")
        total_demand += c[city]
    print(f"\nTotal demand from remaining cities: {total_demand} TB/s")
    print(f"Total switching station capacity: {sum(C.values())} TB/s")
    
    # Calculate distances from remaining cities to switching stations
    d = {}
    for i in R:
        for j in S:
            d[i, j] = euclid(i, j, coords)
    
    # Create Gurobi model
    m = gp.Model("Task2_Switching_Stations")
    m.setParam('OutputFlag', 0)  # Completely disable Gurobi output to avoid encoding issues
    
    # Enable sensitivity analysis
    m.setParam('SolutionLimit', 1)
    
    # Create mapping for clean variable names
    R_clean = {city: clean_name_for_constraint(city) for city in R}
    S_clean = {station: clean_name_for_constraint(station) for station in S}
    
    # Decision variables: Flow from city i to station j (continuous, >= 0)
    # Use clean names for variables too
    Q = {}
    for i in R:
        for j in S:
            Q[i, j] = m.addVar(lb=0, vtype=GRB.CONTINUOUS, 
                              name=f"Q_{R_clean[i]}_{S_clean[j]}")
    
    # Objective: Minimize total cost of laying copper cables
    m.setObjective(
        gp.quicksum(alpha * d[i, j] * Q[i, j] for i in R for j in S),
        GRB.MINIMIZE
    )
    
    # Constraints
    
    # 1. Demand Satisfaction: Each city's requirement must be met exactly
    demand_constrs = {}
    for i in R:
        clean_i = clean_name_for_constraint(i)
        demand_constrs[i] = m.addConstr(
            gp.quicksum(Q[i, j] for j in S) == c[i],
            f"demand_{clean_i}"
        )
    
    # 2. Cable Capacity Limits: Maximum 200 TB/s per cable
    cable_constrs = {}
    for i in R:
        for j in S:
            clean_i = clean_name_for_constraint(i)
            clean_j = clean_name_for_constraint(j)
            cable_constrs[i, j] = m.addConstr(
                Q[i, j] <= beta, 
                f"cable_cap_{clean_i}_{clean_j}"
            )
    
    # 3. Switching Station Capacity
    station_constrs = {}
    for j in S:
        clean_j = clean_name_for_constraint(j)
        station_constrs[j] = m.addConstr(
            gp.quicksum(Q[i, j] for i in R) <= C[j],
            f"station_cap_{clean_j}"
        )
    
    # 4. Shannon's Law: Signal degradation with distance
    shannon_constrs = {}
    for i in R:
        for j in S:
            clean_i = clean_name_for_constraint(i)
            clean_j = clean_name_for_constraint(j)
            shannon_limit = 2 * math.pi * min(35, 70 - d[i, j]) #fasit sier uten min greien
            if shannon_limit > 0:
                shannon_constrs[i, j] = m.addConstr(
                    Q[i, j] <= shannon_limit, 
                    f"shannon_{clean_i}_{clean_j}"
                )
            else:
                # Distance too large, no connection possible
                shannon_constrs[i, j] = m.addConstr(
                    Q[i, j] == 0, 
                    f"shannon_zero_{clean_i}_{clean_j}"
                )
    
    # 5. Łódź Special Constraint: Maximum 40% on any single cable
    lodz_constrs = {}
    if "Łódź" in R:
        for j in S:
            clean_j = clean_name_for_constraint(j)
            lodz_constrs[j] = m.addConstr(
                Q["Łódź", j] <= 0.4 * c["Łódź"],
                f"lodz_limit_{clean_j}"
            )
    
    # Optimize
    print("\n" + "-"*40)
    print("Optimizing...")
    print("-"*40)
    
    # Suppress Gurobi output during optimization
    m.setParam('OutputFlag', 0)
    m.optimize()
    
    # Print optimization status
    if m.status == GRB.OPTIMAL:
        print(f"Optimization successful! Objective: {m.objVal:.2f}")
    else:
        print(f"Optimization status: {m.status}")
    
    # Check solution status
    if m.status != GRB.OPTIMAL:
        print(f"\nOptimization failed. Status: {m.status}")
        if m.status == GRB.INFEASIBLE:
            print("Problem is infeasible. Computing IIS...")
            m.computeIIS()
            m.write("task2_iis.ilp")
            print("IIS written to task2_iis.ilp")
        return None
    
    # Extract solution
    print("\n" + "="*60)
    print("OPTIMAL SOLUTION FOUND")
    print("="*60)
    
    total_cost = m.objVal
    
    # Question 1: Total costs
    print(f"\n1. TOTAL COST TO CONNECT REMAINING CITIES:")
    print(f"   {total_cost:,.2f} EUR")
    
    # Question 2: Poznań and Wałbrzych capacity
    print(f"\n2. CAPACITY INSTALLED FOR POZNAŃ AND WAŁBRZYCH:")
    print("-"*50)
    
    for city in ["Poznań", "Wałbrzych"]:
        if city in R:
            print(f"\n{city}:")
            total_installed = 0
            for j in S:
                flow = Q[city, j].X
                if flow > 0.001:
                    cost = alpha * d[city, j] * flow
                    print(f"  -> {j}: {flow:.2f} TB/s (Cost: {cost:,.2f} EUR)")
                    total_installed += flow
            print(f"  Total installed: {total_installed:.2f} TB/s")
            print(f"  Required: {c[city]:.2f} TB/s")
        else:
            print(f"{city}: On main line (not a remaining city)")
    
    # Question 3: Kielce utilization
    print(f"\n3. KIELCE SWITCHING STATION UTILIZATION:")
    print("-"*50)
    kielce_flow = sum(Q[i, "Kielce"].X for i in R)
    kielce_utilization = (kielce_flow / C["Kielce"]) * 100
    print(f"  Total capacity: {C['Kielce']} TB/s")
    print(f"  Used capacity: {kielce_flow:.2f} TB/s")
    print(f"  Utilization: {kielce_utilization:.2f}%")
    
    # Additional analysis
    print(f"\n4. DETAILED CONNECTION SUMMARY:")
    print("-"*50)
    
    # Station utilization summary
    print("\nSwitching Station Utilization:")
    for j in S:
        total_flow = sum(Q[i, j].X for i in R)
        utilization = (total_flow / C[j]) * 100
        print(f"  {j}: {total_flow:.2f}/{C[j]} TB/s ({utilization:.2f}%)")
    
    # All connections
    print("\nAll Connections (City -> Station: Flow):")
    connections = []
    for i in sorted(R):
        city_connections = []
        for j in S:
            flow = Q[i, j].X
            if flow > 0.001:
                city_connections.append(f"{j}({flow:.1f})")
                connections.append({
                    'City': i,
                    'Station': j,
                    'Flow_TB_s': flow,
                    'Distance': d[i, j],
                    'Cost_EUR': alpha * d[i, j] * flow
                })
        if city_connections:
            print(f"  {i}: {', '.join(city_connections)}")
    
    
    # Łódź special constraint check
    if "Łódź" in R:
        print("\n5. ŁÓDŹ SPECIAL CONSTRAINT (40% max per cable):")
        print("-"*50)
        for j in S:
            flow = Q["Łódź", j].X
            if flow > 0.001:
                percentage = (flow / c["Łódź"]) * 100
                print(f"  Łódź -> {j}: {flow:.2f} TB/s ({percentage:.1f}% of total)")
    if m.status == GRB.OPTIMAL:
        print(f"Optimization successful! Objective: {m.objVal:.2f}")

    
    # Save results to CSV
    df = pd.DataFrame(connections)
    df.to_csv('task2_results.csv', index=False)
    print("\n6. RESULTS SAVED TO FILES:")
    print("  - task2_results.csv (connection details)")
    
    # Export model for further analysis
    m.write("task2_model.lp")
    m.write("task2_solution.sol")

    
    return {
        'model': m,
        'total_cost': total_cost,
        'connections': connections,
        'variables': Q
    }
    

if __name__ == "__main__":
    # Run Task 2 optimization
    results = solve_task2_switching_stations()
    

TASK 2: CONNECTING REMAINING CITIES TO SWITCHING STATIONS

Total cities: 26
Cities on main line: 14
Remaining cities to connect: 12

Main line cities: ['Bielsko', 'Gliwice', 'Jastrzębie-Zdrój', 'Jelenia Góra', 'Kalisz', 'Kielce', 'Kraków', 'Legnica', 'Leszno', 'Opole', 'Piotrków', 'Radom', 'Skierniewice', 'Warsaw']

Remaining cities: ['Bytom', 'Czestochowa', 'Katowice', 'Konin', 'Poznań', 'Sieradz', 'Sosnowiec', 'Wałbrzych', 'Wodzisław Śląski', 'Wrocław', 'Zielona Góra', 'Łódź']
Switching stations: ['Kielce', 'Opole', 'Warsaw']

Capacity requirements for remaining cities:
  Bytom: 163 TB/s
  Czestochowa: 278 TB/s
  Katowice: 263 TB/s
  Konin: 236 TB/s
  Poznań: 251 TB/s
  Sieradz: 178 TB/s
  Sosnowiec: 143 TB/s
  Wałbrzych: 129 TB/s
  Wodzisław Śląski: 119 TB/s
  Wrocław: 133 TB/s
  Zielona Góra: 121 TB/s
  Łódź: 317 TB/s

Total demand from remaining cities: 2331 TB/s
Total switching station capacity: 2400 TB/s

----------------------------------------
Optimizing...
-------------------

In [36]:

def show_konin_sensitivity(results):
    m = results['model']
    con = m.getConstrByName("demand_Konin")
    if con is None:
        print("No constraint found for Konin (maybe it's on the main line).")
        return

    print("Sensitivity analysis Konin demand")
    print(f"Shadow price: {con.Pi:.4f}")
    print(f"Lower limit: {con.SARHSLow:.4f}")
    print(f"Upper limit: {con.SARHSUp:.4f}")

show_konin_sensitivity(results)


Sensitivity analysis Konin demand
Shadow price: 37865.4043
Lower limit: 200.0000
Upper limit: 295.1358
